In [1]:
# install pytorch
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117


In [2]:
# install transformers
!pip install transformers

In [25]:
import os, io
from google.cloud import vision
from google.cloud.vision_v1 import types
from google.cloud import translate_v2 as translate
import pandas as pd

In [26]:
# credential_path = "C:/Users/zrowx/OneDrive/바탕 화면/python/VisionAPI/ServiceAccountToken.json"
credential_path = "ServiceAccountToken.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path

In [27]:
client = vision.ImageAnnotatorClient()
trans_client = translate.Client()
path = 'python_kor.jpg' 
with io.open(path, 'rb') as image_file:
    content = image_file.read()

In [28]:
image = vision.Image(content=content)

In [29]:
response = client.text_detection(image=image)
texts = response.text_annotations

In [30]:
df = pd.DataFrame(columns=['locale', 'description'])
for text in texts:
        df = df.append(
                dict(
                        locale=text.locale,
                        description=text.description
                ),
                ignore_index=True
        )

In [31]:
# ocr complete
text = df['description'][0]

In [32]:
# lang detect
detected_lang = trans_client.detect_language(text).get('language')

In [33]:
# print(detected_lang)

In [34]:
# translate into english
translated = trans_client.translate(df['description'][0], target_language='en')

In [35]:
translated = translated.get('translatedText')

In [36]:
# print(translated)

In [37]:
# importing dependencies from transformers
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [38]:
# load tokenizer
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")

In [39]:
# load model
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")

In [40]:
# create tokens - number representation of the text
tokens = tokenizer(translated, truncation=True, padding="longest", return_tensors="pt")

In [41]:
# summarize
summary = model.generate(**tokens)

In [42]:
# decode summary
result = tokenizer.decode(summary[0])

In [43]:
result = result.strip("<pad>")
result = result.strip("</s>")

In [44]:
# print(result)

In [45]:
final_output = trans_client.translate(result, target_language=detected_lang).get('translatedText')

In [46]:
print(final_output)

파이썬은 프로그래밍 언어입니다.
